### Approaches to Solving MDPs I: Dynamic Programming

### University of Virginia
### Reinforcement Learning
#### Last updated: January 24, 2025

---

### SOURCES 

- Reinforcement Learning, RS Sutton & AG Barto, 2nd edition. Chapter 4
- Mastering Reinforcement Learning with Python, Enes Bilgin. Chapter 5

### LEARNING OUTCOMES

- Understand the conceptual ideas behind dynamic programming 
- Study the Bellman optimality equations for estimating the optimal value functions
- Discuss the strengths and limitations of Dynamic Programming
- Understand and apply the value iteration algorithm
- Explain how generalized policy iteration works to find optimality

### CONCEPTS

- Dynamic Programming
- Value Iteration
- Policy Iteration
- Generalized Policy Iteration
- Bellman optimality equations


#### There are three important approaches to solving MDPs:

- Dynamic Programming (DP)
- Monte Carlo Simulation (MC)
- Temporal Difference (TD)

**DP methods**  
Provide optimal solutions to Markov decision processes (MDPs)  
Much more efficient than exhaustive search  
Developed by Richard Bellman in the 1950s  

Strengths: 

- Closed-form equations for optimality
- Intuitive, as it exploits the property of overlapping substructure
- Efficient computation

Limitations:

- Requires complete knowledge of the state transition and reward dynamics of the environment. This is often unrealistic.
- When the state space is continuous or massive, the required runtime becomes infeasible

**MC methods**  
Model-free method that uses sampled transitions from the environment, so transition matrix (dynamics of system) is not required

An update is made after a full trajectory is simulated

Relies on convergence properties when measuring value functions

**TD methods**  
Model-free method that learns by bootstrapping from the current estimate of the value function. 

Samples from the environment like Monte Carlo, and performs updates based on current estimates, like dynamic programming.

An update is made after each time step is simulated (learns faster than MC)

In this notebook we will concentrate on Dynamic Programming.

---  

#### Motivating the Dynamic Programming Method

We will only briefly touch on DP: it is very instructive but generally impractical for large-scale problems.

Consider this example where a robot needs to decide the best path to traverse on a binary tree. 

At each time step $t$, the robot can move UP or DOWN (actions).

This tree has three layers (ignoring the start, or root, node).
Rewards are located on each node.

We make this problem a little easier with these features:
- discount factor will be $\gamma=1$ (no discounting)
- the robot decides which path to take (the action) with probability 1; deterministic policy $\pi(s)$
- the environment is deterministic: the robot will move in the intended direction

Task: select the path that maximizes expected return. What is the path and return?  

Is this problem Markov?

**Example: Robot maximization task**

![tree_task1](./tree_task1_binary.png)

Working from left to right is a brute force method that solves the problem. 

Brute force -> redundant calculations. Is there a more efficient way?



Hint: Think about working backwards from right to left, backing the rewards to earlier layers in the tree.

[enter your ideas here] 

Starting at the next-to-last layer, it will look ahead from each node to the future rewards.

It will select the next step with highest reward, acting greedily.  It can compute this maximum before taking a step.


![tree_task1](./tree_task1_working_backwards_binary.png)

For example at the top node with reward 2, it would select the path leading to 3.  
The value of the top node with reward 2 is then 2 + 3 = 5. It would receive immediate reward 2 and would receive 3 at the next step.

Let's back up the rewards earlier in the tree and update the values at layer 2.

![tree_task1](./tree_task1_working_backwards_adding_rewards1_binary.png)

Notice the structure: use the potential values in the next step $t+1$ to compute the value at the earlier step $t$

This is iterative and can be easily coded for massive trees

In general, **problems that have overlapping substructure are good candidates for dynamic programming solutions**

Repeat this procedure, working backwards through the tree and pushing values earlier

![tree_task1](./tree_task1_working_backwards_adding_rewards2_binary.png)

![tree_task1](./tree_task1_working_backwards2_adding_rewards2_binary.png)

This produces the best path (in green) and maximum reward of 11.

**Exercise**
A short exercise is found in *exercises_robot.ipynb* 

---

**Relationship to Pricing an American Call Option**  

If you have priced options on a binomal lattice, you will notice similar ideas being applied.  

A *call option* on a stock allows the holder to buy the stock (*exercise* the option) for a fixed price (*strike price*) by a fixed date (*expiry*).    
An *American* call option allows the holder to exercise before expiry.  He will take the action - *(exercise, no exercise)* - that maximizes value.  

For the option pricing problem, it is a bit more involved than the robot problem:

- there will be discounting to reflect time value of money
- the environment is stochastic: the stock price will fluctuate with a random component

---

#### Key Formulas in Dynamic Programming

The formulas will handle the general case to include discounting, a policy that follows a distribution, and uncertainty in the environment.

Here are the Bellman optimality equations for computing optimal state value $v_*(s)$ and optimal action value $q_*(s,a)$

$v_*(s) = \underset{a}{\operatorname{\max}} \mathbb{E} [R_{t+1} + \gamma v_*(S_{t+1}) | S_t=s]$


$q_*(s,a) = \mathbb{E} [R_{t+1} + \gamma \underset{a'}{\operatorname{\max}} q_*(S_{t+1},a') | S_t=s, A_t=a]$

For $v_*(s)$, the agent selects actions that maximize the state value for each state *s*.

For $q_*(s,a)$, the agent selects action *a* from state *s* and follows the optimal policy afterwards. 

In order to compute expectations, the transition probabilities need to be known in advance.

Additionally, the expectation calculation grows with the number of states.

Notice the structure: the Bellman equations are recursive. They can be easily programmed.

#### Policy Evaluation and Improvement

An important goal is to discover the optimal policy.  
This can be done by measuring the value of the current policy, trying to improve the policy, and repeating these two steps.

When environment dynamics are known, the Bellman equation can be solved iteratively.  
The initial value estimates $v_0$ are chosen arbitrarily; in the terminal state, values are zero.

The Bellman equation is turned into an update step.

The *policy iteration* algorithm can be found in Sutton & Barto, but it is computationally expensive.  
A shortcut method is *value iteration*.

### Value Iteration

The idea of value iteration is to refine estimates of the value function through the Bellman equation.  
We start with random $V(s)$, we update with the action that maximizes the expected value, and we repeat until a criterion is met.

Once we have estimates for $V(s)$ for all $s$, the policy will select actions that maximize expected value.  
It can be greedy since all values reflect expected cumulative gains.


![value_iteration](./value_iteration.png)

### Generalized Policy Iteration (GPI)

**Policy iteration** consists of two simultaneous, interacting processes: 
- *policy evaluation*: make the value function consistent with the current policy 
- *policy improvement*: make the policy greedy given the current value function

The graphic below demonstrates this idea, using iteration to build a better policy and then value the updated policy.

![gpi](./gpi1.png)

Nearly all RL problems are described by GPI.

The two steps work in opposition to find the optimal value function and optimal policy.

### Next Steps

Since we often won't have the transition probabilities to solve real-world problems, we will need other methods.

Next, we will study a technique that sidesteps the need for transition probabilities: Monte Carlo simulation.

---